# Link prediction

This is an example from the social networking course. The description of the dataset is pretty bad, as they have one dataset named `future_predictions` which is of the form:

```python
Index        "Future Prediction"
(src, dst)      0, 1, or np.nan
```

where
* 0 means there will not be an edge in future
* 1 means there will be an edge in future
* .... and np.nan means it is unknown (i.e. it is part of the test set)

This is really silly, because we don't know any edges in the future. 

## Better approach (for your project)

Let's look at the 1000 nodes in the graph, and the 1000 * 1000 ~ 1 million possible directed edges. 

In your projet, take the ~1000 nodes and potential 1 million edges. For each (src, dst) pair, put a 1 if the edge exists and a zero if it doesn't. 

Then split the nodes up using a train/test split. Try to stratify (i.e. same proportion of no links for training and testing sets).

Then make a subgraph containing only the edges present in the training set (these are the "blue edge", modelling the state of the network some time in the past). Use these to construct the graph features, because you can only use current knowledge about the graph.

When evaluating the test set, you are still only allowed to use the features derived from the edges in the training set.

The setup is:
- we know the network "in the future"
- we copy down some of the info about the network in the future
- we ignore other information (i.e. the test set), and try to predict it.

If our predictions are reasonably good, then we have some trust that we can extend this to the future by picking nodes that don't currently have an edge, but trying to predict whether or not they will.

In the assignment, the test train split is done for us (i.e. if the edge is np.nan, it is part of the test set).

In [1]:
import pandas as pd
import networkx as nx
import pickle
import numpy as np

whole_graph = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval}).rename(columns={"Future Connection":"connected"})
whole_graph.head()

,connected
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0


In [2]:
# class imbalance?
whole_graph['connected'].value_counts()

0.0    337002
1.0     29332
Name: connected, dtype: int64

In [3]:
# What are the existing connections in the training set?
# This gives us the 0/1 edges from the graph
train_graph_frame = whole_graph[~whole_graph['connected'].isnull()]

# use this to construct the graph, by just selecting the edges with ones
edges_that_exist = train_graph_frame[whole_graph['connected']==1]

train_graph = nx.Graph()
train_graph.add_edges_from(edges_that_exist.index)

/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [4]:
# how big is the graph?
message = f"""There are {len(train_graph.nodes())} nodes and  {len(train_graph.edges())} edges"""
print(message)

There are 1005 nodes and  29332 edges


## Make graph features

We are going to make features from the existing (training) graph only, because that is all we have information on. First, the long way:

In [5]:
jaccard = {}
for src, dst, score in nx.jaccard_coefficient(train_graph): # note this only calculates scores for non-existent edges
    jaccard[(src,dst)] = score

jaccard_frame = pd.DataFrame.from_dict(jaccard, orient='index')
jaccard_frame.head()

,0
"(0, 1)",0.153153
"(0, 2)",0.044444
"(0, 3)",0.066667
"(0, 4)",0.063584
"(0, 5)",0.041667


Now the way that uses a function to get the metrics:

In [6]:
def make_measure_frame(graph, metric_function, node_pair_list):
    '''
    graph: a networkx graph
    metric_function: a function that scores edges (e.g. nx.jaccard_coefficient)
    
    returns:
    A dataframe where the index is the edge (src, dst) and the column is the metric value
    '''
    metric = {(src, dst): score for src, dst, score in metric_function(graph, node_pair_list)}
    metric_df = pd.DataFrame.from_dict(metric, orient='index')
    return metric_df

In [7]:
def make_features(train_graph, node_list):
    jaccard_frame = make_measure_frame(train_graph, nx.jaccard_coefficient, node_list)
    resource_frame= make_measure_frame(train_graph, nx.resource_allocation_index, node_list)
    adamic_frame = make_measure_frame(train_graph, nx.adamic_adar_index, node_list)
    
    edge_features = pd.concat([jaccard_frame, resource_frame, adamic_frame], axis=1)
    edge_features.columns=['jaccard', 'resource', 'adamic']
    return edge_features

In [8]:
edge_features = make_features(train_graph, node_list = list(train_graph_frame.index))
edge_features.head()

,jaccard,resource,adamic
"(6, 840)",0.096154,0.118041,3.038920
"(4, 197)",0.089431,0.157033,4.382085
"(620, 979)",0.032258,0.013333,0.231616
"(519, 872)",0.071429,0.029998,0.816177
"(382, 423)",0.021505,0.011757,0.387085


In [9]:
## Turns out that pandas gets confused by tuple indexing.
## Instead, it prefers multi-index columns. Instead of messing with that, use pd.merge so we are 
## only selecting the features that belong to the edges in the training set (either the 0s or 1s)
graph_info = pd.merge(train_graph_frame, edge_features, how='left', left_index=True, right_index=True)
X_train = graph_info.iloc[:, 1:]
y_train = graph_info.iloc[:, 0]

X_train.head()

,jaccard,resource,adamic
"(6, 840)",0.096154,0.118041,3.038920
"(4, 197)",0.089431,0.157033,4.382085
"(620, 979)",0.032258,0.013333,0.231616
"(519, 872)",0.071429,0.029998,0.816177
"(382, 423)",0.021505,0.011757,0.387085


## Link prediction (classifier)

Now introduce the classifier. Note there is a huge cheat going on with cross-validation. When doing cross validation, we should be doing it manually because we only have access to the graph infomation of the piece we are training on. This makes CV a lot slower. 

So there is some laziness/expediency taken in the steps below.

To do it properly:
1. For each fold, find the graph using only the known part of the fold
2. Generate the features for that fold
3. Test on the hold out fold

You would use KFolds, because constructing the graph is something that would be done manually.

In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score

In [11]:
params = {
    'C': np.logspace(-9,6, 20)
}

grid_logistic = GridSearchCV(LogisticRegression(), params, cv=5)

grid_logistic.fit(X_train, y_train)
y_score = grid_logistic.predict_proba(X_train)[:, 1]
auc_score = roc_auc_score(y_train, y_score)

msg = f"""
Accuracy from logistic regression is {grid_logistic.best_score_}, and an AUC score of {auc_score}
"""
print(msg)


Accuracy from logistic regression is 0.9420774484486835, and an AUC score of 0.8904156471291396



In [12]:
params = {
    'max_depth': [2, 4, 6],
    'n_estimators': [20, 50, 100, 200]
}

grid_forest = GridSearchCV(RandomForestClassifier(), params, cv=5)

grid_forest.fit(X_train, y_train)
y_score = grid_forest.predict_proba(X_train)[:, 1]
auc_score = roc_auc_score(y_train, y_score)

msg = f"""
Accuracy is {grid_forest.best_score_}, and an AUC score of {auc_score}
"""
print(msg)

KeyboardInterrupt: 

^^ This cell takes a long time (~5 minutes) to run, exited prematurely

## Making the test set

In [13]:
test_node_list = list(whole_graph[whole_graph.isnull()].index)
best_classifier = grid_logistic

In [14]:
# Still use the training graph
X_test = make_features(train_graph, test_node_list)
predict_scores = best_classifier.predict_proba(X_test)[:, 1]

In [15]:
predict_scores[:5]

array([0.11482894, 0.17243506, 0.02277958, 0.03194493, 0.02067701])